# Video stream train

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path

from keras.layers.core import Dense,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [2]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [106]:
path = "Datasets/RAVDESS_frames/"

img_height = 224
img_width = 224
batch_size = 64
num_classes = 8

emotions = {1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}
emotional_intensity = {1:'normal', 2:'strong'}

num_frames_desired = 23     #number of frames per clip in order to have balanced classes

Train set load

In [113]:
filenames = []
for (dirpath, dirnames, fn) in os.walk(path):
    if fn != []:
        if 'Actor_23' not in dirpath and 'Actor_24' not in dirpath:
            o = [fn[0].split('-')[2]] * len(fn)
            filenames.append(list(zip(fn, o)))

In [98]:
min([len(elem) for elem in filenames])

23

In [110]:
def sampling(list, num_frames_desired):
    tot = []
    for elem in list:
        sampled_list = random.sample(elem, num_frames_desired)
        tot += sampled_list
    return(tot)

def parse_image(filename):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [img_height, img_width])
#     image = tf.image.resize(image, [img_height, img_width])
    print(image.shape)
    return image

def configure_for_performance(ds):
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.repeat()
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

In [50]:
filenames_sampled = sampling(filenames, num_frames_desired)

random.shuffle(filenames_sampled)

zipped = [list(t) for t in zip(*filenames_sampled)]

filenames = zipped[0]
labels = zipped[1]

filenames_ds = tf.data.Dataset.from_tensor_slices(filenames)
images_ds = filenames_ds.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
labels_ds = tf.data.Dataset.from_tensor_slices(labels)
ds = tf.data.Dataset.zip((images_ds, labels_ds))
val_ds = configure_for_performance(ds)

(224, 224, 3)


In [52]:
frame_number_train = len(filenames)
step_per_epoch_train = frame_number_train // batch_size
print(step_per_epoch_train)

621
